In [1]:
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from scipy import stats
import seaborn as sns

presurvey1 = "../Data/Fall 2019/CSE 8A Fall 2019 DeID/presurvey.csv"
gradebook1 = "../Data/Fall 2019/CSE 8A Fall 2019 DeID/gradebook.csv"
presurvey2 = "../Data/Fall 2019/CSE 11 Fall 2019 DeID/presurvey.csv"
gradebook2 = "../Data/Fall 2019/CSE 11 Fall 2019 DeID/gradebook.csv"
presurvey3 = "../Data/Fall 2019/CSE 12 Fall 2019 DeID/presurvey.csv"
gradebook3 = "../Data/Fall 2019/CSE 12 Fall 2019 DeID/gradebook.csv"

In [2]:
reflections2 = []
for i in range (1,9):
    reflections2.append(pd.read_csv("../Data/Fall 2019/CSE 11 Fall 2019 DeID/R" +str(i)+".csv"))

In [3]:
reflections3 = []
for i in range (1,11):
    reflections3.append(pd.read_csv("../Data/Fall 2019/CSE 12 Fall 2019 DeID/R" +str(i)+".csv"))

In [4]:
reflections1 = []
for i in range (1,9):
    reflections1.append(pd.read_csv("../Data/Fall 2019/CSE 8A Fall 2019 DeID/R" +str(i)+".csv"))

In [5]:
def scaled_categories(reflections, presurvey, category, label, x):
    categoryIssue = 0
    for i in range(x):
        category_issue = reflections[i][category]
        DeID = reflections[i]["DeID"]
        s = label + str(i) 
        cat = DataFrame({"DeID": DeID, s: category_issue})
        if i == 0:
            categoryIssue = cat
        else:
            categoryIssue = pd.merge(categoryIssue, cat, on="DeID", how="outer")
            
    categoryIssue = categoryIssue.drop_duplicates(subset=['DeID'])     
    df = pd.read_csv(presurvey)
    status = df["Did.you.enter.UCSD.as.a.transfer.student.from.another..2.year.or.4.year..college.or.university."] == "Yes"
    DeID = df["DeID"]
    transferStatus = DataFrame({"DeID": DeID, "Transfer Status": status})

    mergedData = pd.merge(transferStatus, categoryIssue, on="DeID")
    transfer = mergedData[mergedData["Transfer Status"] == True]
    native = mergedData[mergedData["Transfer Status"] == False]
    
    return {'transfer' : transfer, 'native' : native}
    

In [11]:
def RemovedOutlier(transfer, native, x, x_label):
    transferIssues = []
    nativeIssues = []
    transferMedian = []
    nativeMedian = []
    transferStd = []
    nativeStd = []
    transferLen = []
    nativeLen = []
    transferMax = []
    nativeMax = []
    transferMin = []
    nativeMin = []
    utestStatistics = []
    pValuesU = []
    TQUL = []
    TQLL = []
    NQUL = []
    NQLL = []

    for i in range(x):
        df = pd.DataFrame({label+str(i):transfer[label+str(i)],"DeID":transfer["DeID"]})
        df = df.drop_duplicates(subset=["DeID"])
        df = df.dropna(axis=0)
        l = len(df)
        Q1 = df[label+str(i)].quantile(0.05)
        Q3 = df[label+str(i)].quantile(0.95)
        IQR = Q3 - Q1    #IQR is interquartile range. 
        QLL = Q1 - 1.5 * IQR
        QUL = Q3 + 1.5 * IQR
        
        TQUL.append(QUL)
        TQLL.append(QLL)
        
        filterInclude = (df[label+str(i)] >= QLL) & (df[label+str(i)] <= QUL)   
        filterExclude = (df[label+str(i)] < QLL) | (df[label+str(i)] > QUL)
        
        df = df.loc[filterInclude]
        #print("Removed %: " + str((l - len(df))*100/l))
        transferRefined = df

        df = pd.DataFrame({label+str(i):native[label+str(i)], "DeID":native["DeID"]})
        df = df.drop_duplicates(subset=["DeID"])
        df = df.dropna(axis=0)       
        l = len(df)
        Q1 = df[label+str(i)].quantile(0.05)
        Q3 = df[label+str(i)].quantile(0.95)
        IQR = Q3 - Q1    #IQR is interquartile range. 
        QLL = 0
        QUL = Q3 + 1.5 * IQR
        
        NQUL.append(QUL)
        NQLL.append(QLL)
        
        filterInclude = (df[label+str(i)] >= QLL) & (df[label+str(i)] <= QUL)   
        filterExclude = (df[label+str(i)] < QLL) | (df[label+str(i)] > QUL)
        
        df = df.loc[filterInclude]
        #print("Removed %: " + str((l - len(df))*100/l))
        nativeRefined = df
        

        transferIssues.append(transferRefined[label+str(i)].mean())
        nativeIssues.append(nativeRefined[label+str(i)].mean())

        transferMedian.append(transferRefined[label+str(i)].median())
        nativeMedian.append(nativeRefined[label+str(i)].median())

        transferStd.append(transferRefined[label+str(i)].std())
        nativeStd.append(nativeRefined[label+str(i)].std())

        transferLen.append(len(transferRefined[label+str(i)]))
        nativeLen.append(len(nativeRefined[label+str(i)]))
        
        transferMax.append(transferRefined[label+str(i)].max())
        nativeMax.append(nativeRefined[label+str(i)].max())
        
        transferMin.append(transferRefined[label+str(i)].min())
        nativeMin.append(nativeRefined[label+str(i)].min())

        ttest_results = stats.ttest_ind(transferRefined[label+str(i)], nativeRefined[label+str(i)])
        utest_results = stats.mannwhitneyu(transferRefined[label+str(i)], nativeRefined[label+str(i)], use_continuity=False)
        utestStatistics.append(utest_results[0])
        pValuesU.append(utest_results[1])

    df = DataFrame({"Week" : Series(x_label), 
                    "Transfer Mean" : Series(transferIssues), 
                    "Native Mean" : Series(nativeIssues),
                    "Transfer Std Dev" : Series(transferStd),
                    "Native Std Dev" : Series(nativeStd),
                    "Transfer Max" : Series(transferMax),
                    "Native Max" : Series(nativeMax),
                    "Transfer Min" : Series(transferMin),
                    "Native Min" : Series(nativeMin),
                    "Transfer Number" : Series(transferLen),
                    "Native Number" : Series(nativeLen),
                    "T Upper Limit" : TQUL,
                    "N Upper Limit" : NQUL,
                    "p-value (u-test)" : pValuesU,
                    "Test Statistic (u-test)" : utestStatistics})
    return df

In [12]:
category = "At.this.time..approximately.how.many.other.students.in.this.course.would.you.be.comfortable.reaching.out.to.study.with."
label = "# of People "
graphTitle = "# of People you Feel Comfortable Reaching Out To"
X_title = "Week"
Y_title = "Number of Students"
x = 8
scale1 = "1"
scale5 = "5"

data = scaled_categories(reflections1, presurvey1, category, label, 8)
transfer1 = data['transfer']
native1 = data['native']

data = scaled_categories(reflections2, presurvey2, category, label, 8)
transfer2 = data['transfer']
native2 = data['native']

x = 10
data = scaled_categories(reflections3, presurvey3, category, label, 10)
transfer3 = data['transfer']
native3 = data['native']


In [13]:
x_label1 = ["Week 2", "Week 3", "Week 4", "Week 5", "Week 6", "Week 7", "Week 8", "Week 9"]
x_label2 = ["Week 1", "Week 2", "Week 3", "Week 4", "Week 5", "Week 6", "Week 7", "Week 8", "Week 9", "Week 10"]

df1 = RemovedOutlier(transfer1, native1, 8, x_label1)
df2 = RemovedOutlier(transfer2, native2, 8, x_label1)
df3 = RemovedOutlier(transfer3, native3, 10, x_label2)

In [14]:
display("CS 1:")
display(df1)
display("CS 1A:")
display(df2)
display("CS 2:")
display(df3)

'CS 1:'

,Week,Transfer Mean,Native Mean,Transfer Std Dev,Native Std Dev,Transfer Max,Native Max,Transfer Min,Native Min,Transfer Number,Native Number,T Upper Limit,N Upper Limit,p-value (u-test),Test Statistic (u-test)
0,Week 2,2.857143,3.099476,3.882027,2.837793,20.0,20.0,0.0,0.0,56,382,50.00,25.000,0.019151,8891.5
1,Week 3,8.480000,3.557789,42.188711,3.334489,300.0,20.0,0.0,0.0,50,398,365.00,25.000,0.001990,7495.0
2,Week 4,2.560000,3.738693,3.375542,3.231935,20.0,25.0,0.0,0.0,50,398,39.00,25.000,0.000029,6518.0
3,Week 5,2.066667,4.065617,1.788854,3.693294,10.0,30.0,0.0,0.0,45,381,12.50,30.000,0.000003,5051.5
4,Week 6,2.596154,4.031746,2.225165,3.408376,10.0,20.0,0.0,0.0,52,378,17.50,28.500,0.000299,6981.5
5,Week 7,3.282609,4.546154,4.042348,4.159626,20.0,30.0,0.0,0.0,46,390,42.50,36.000,0.001314,6563.5
6,Week 8,2.277778,4.377841,1.891501,3.680528,10.0,20.0,0.0,0.0,36,352,12.50,23.800,0.000022,3741.0
7,Week 9,2.363636,4.532338,1.954017,4.050824,10.0,30.0,0.0,0.0,44,402,23.75,32.625,0.000011,5433.0


'CS 1A:'

,Week,Transfer Mean,Native Mean,Transfer Std Dev,Native Std Dev,Transfer Max,Native Max,Transfer Min,Native Min,Transfer Number,Native Number,T Upper Limit,N Upper Limit,p-value (u-test),Test Statistic (u-test)
0,Week 2,1.969697,5.293160,2.007561,6.198148,10.0,50.0,0.0,0.0,33,307,16.875,50.000,1.819835e-06,2602.5
1,Week 3,2.000000,5.430976,1.460593,5.472547,5.0,49.0,0.0,0.0,31,297,11.125,49.775,6.324427e-07,2189.5
2,Week 4,2.000000,5.980392,1.388730,6.810764,5.0,50.0,0.0,0.0,29,306,11.375,50.000,5.655545e-07,2029.0
3,Week 5,2.120000,5.806667,1.536229,4.974445,5.0,30.0,0.0,0.0,25,300,12.500,48.500,1.396050e-06,1651.0
4,Week 6,2.160000,5.939597,1.624808,4.919867,5.0,30.0,-1.0,0.0,25,298,12.500,48.500,1.298787e-06,1631.5
5,Week 7,2.033333,6.102990,1.731719,4.961790,6.0,30.0,-2.0,0.0,30,301,13.750,49.550,1.703095e-08,1776.5
6,Week 8,2.218750,6.015723,1.913186,5.014465,7.0,30.0,-3.0,0.0,32,318,14.500,48.500,1.670693e-07,2322.5
7,Week 9,3.034483,6.530686,2.993836,6.714949,10.0,50.0,-3.0,0.0,29,277,25.000,50.000,8.661514e-05,2324.5


'CS 2:'

,Week,Transfer Mean,Native Mean,Transfer Std Dev,Native Std Dev,Transfer Max,Native Max,Transfer Min,Native Min,Transfer Number,Native Number,T Upper Limit,N Upper Limit,p-value (u-test),Test Statistic (u-test)
0,Week 1,2.692308,2.460317,2.421622,1.861830,10.0,10.0,0.0,0.0,52,126,25.000,17.5,0.450309,3237.5
1,Week 2,2.607143,2.666667,2.269790,2.215647,10.0,10.0,0.0,0.0,56,144,25.000,25.0,0.329362,3872.0
2,Week 3,3.172414,2.913333,3.266727,2.490079,20.0,12.0,0.0,0.0,58,150,25.000,25.0,0.380754,4233.5
3,Week 4,4.803922,3.145985,9.316694,2.764109,50.0,12.0,0.0,0.0,51,137,78.125,25.0,0.498783,3492.5
4,Week 5,3.500000,3.134454,4.670237,2.878690,25.0,15.0,0.0,0.0,46,119,42.500,29.5,0.404021,2671.0
5,Week 6,4.229167,3.406504,5.207358,3.117474,25.0,20.0,0.0,0.0,48,123,50.000,28.5,0.443957,2911.5
6,Week 7,4.108696,3.433824,5.470033,3.208097,30.0,20.0,0.0,0.0,46,136,47.000,25.5,0.481048,3113.5
7,Week 8,7.428571,3.047059,11.780173,2.882236,50.0,15.0,0.0,0.0,28,85,75.000,25.0,0.080717,982.0
8,Week 9,3.076923,3.305263,2.381984,3.582088,10.0,20.0,0.0,0.0,26,95,23.500,37.5,0.311396,1158.0
9,Week 10,4.081081,3.726415,3.729599,3.597706,16.0,20.0,0.0,0.0,37,106,36.375,25.0,0.323402,1862.5


In [15]:
fig = go.Figure(layout=go.Layout(yaxis=dict(range=[0,6.25])))

fig.add_trace(go.Scatter(x=x_label2, y=df3["Transfer Mean"],
                    mode='lines+markers',
                    name='Transfer CS 2',
                    marker_symbol="square",
                    marker=dict(
                        color='#69c6af',
                        size=5,
                        line=dict(
                                color='#52bca3',
                                width=5
                            )
                        )
               ))
fig.add_trace(go.Scatter(x=x_label2, y=df3["Native Mean"],
                    mode='lines+markers',
                    name='Native CS 2',
                    marker_symbol="circle-dot",
                    marker=dict(
                        color='#69c6af',
                        size=5,
                        line=dict(
                                color='#52bca3',
                                width=5
                            )
                        )
               ))

fig.add_trace(go.Scatter(x=x_label1, y=df2["Transfer Mean"],
                    mode='lines+markers',
                    name='Transfer CS 1A',
                    marker_symbol="square",
                    marker=dict(
                        color='#9a61b0',
                        size=5,
                        line=dict(
                                color='#5d69b1',
                                width=5
                            )
                        )
               ))
fig.add_trace(go.Scatter(x=x_label1, y=df2["Native Mean"],
                    mode='lines+markers',
                    name='Native CS 1A',
                    marker_symbol="circle-dot",
                    marker=dict(
                        color='#9a61b0',
                        size=5,
                        line=dict(
                                color='#5d69b1',
                                width=5
                            )
                        )
               ))

fig.add_trace(go.Scatter(x=x_label1, y=df1["Transfer Mean"],
                    mode='lines+markers',
                    name='Transfer CS 1',
                    marker_symbol="square",
                    marker=dict(
                        color='#c7a302',
                        size=5,
                        line=dict(
                                color="#e58606",
                                width=5
                            )
                        )
             ))
fig.add_trace(go.Scatter(x=x_label1, y=df1["Native Mean"],
                    mode='lines+markers',
                    name='Native CS 1',
                    marker_symbol="circle-dot",
                    marker=dict(
                        color='#c7a302',
                        size=5,
                        line=dict(
                                color='#e58606',
                                width=5
                            )
                        )
               ))


fig.update_layout(
                  xaxis_title="<b>Week</b>",
                  height=525,
                  width = 950,
                  yaxis_title="<b>Number of Students</b>",
                  legend_title="<b>Legend</b>",
                 )


fig.show()